# ***Finding Case Outcomes and Labelling Data***

In this notebook, i aim to try and isolate case outcomes from the documents.csv
file manually, as well as using the RECAP open sources PACER data.

For missing outcomes that I fail to match, it might be useful idea to train LegalBERT on a downstream classification task using the labelled data and then using predicted case outcomes in my regressions.

In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#loading up google drive to access csv files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#loading my csv file
documents = pd.read_csv('/content/drive/MyDrive/ECO225_Final_Project/PART1/documents.csv')

<ipython-input-2-76cc3cccd0bc>:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  documents = pd.read_csv('/content/drive/MyDrive/ECO225_Final_Project/PART1/documents.csv')


In [3]:
#isolating relavent variables
clean_docs = documents.copy()
clean_docs = clean_docs.drop(columns=['case_number_raw', 'attachment', 'doc_date_filed', 'doc_date_uploaded', 'document_url'])

#creating column that replaces NaN long_description values with short_description's
clean_docs['combined_description'] = clean_docs['long_description'].fillna(clean_docs['short_description'])


#dropping all missiing long_descriptions
clean_docs = clean_docs.dropna(subset=['combined_description'])

In [4]:
# Set the display option to show full column contents
pd.set_option('display.max_colwidth', None)

clean_docs.head()

,case_row_id,case_number,district_id,doc_count,doc_number,short_description,long_description,combined_description
0,1.0,0:79-cv-06704,flsd,1,37.0,NaN,"COPY OF PAPER DOCKET SHEET (kw, Deputy Clerk) (Entered: 08/03/2000)","COPY OF PAPER DOCKET SHEET (kw, Deputy Clerk) (Entered: 08/03/2000)"
1,1.0,0:79-cv-06704,flsd,2,NaN,NaN,"CASE CLOSED. Case and Motions no longer referred to Magistrate. (kw, Deputy Clerk) (Entered: 08/03/2000)","CASE CLOSED. Case and Motions no longer referred to Magistrate. (kw, Deputy Clerk) (Entered: 08/03/2000)"
2,3.0,0:83-cv-06860,flsd,1,123.0,NaN,COPY OF PAPER DOCKET SHEET (Former Deputy Clerk) (Entered: 02/13/2004),COPY OF PAPER DOCKET SHEET (Former Deputy Clerk) (Entered: 02/13/2004)
3,3.0,0:83-cv-06860,flsd,2,NaN,NaN,Case closed (Former Deputy Clerk) (Entered: 03/05/1992),Case closed (Former Deputy Clerk) (Entered: 03/05/1992)
4,3.0,0:83-cv-06860,flsd,3,NaN,NaN,Case reopened (Former Deputy Clerk) (Entered: 05/13/1991),Case reopened (Former Deputy Clerk) (Entered: 05/13/1991)


---
# STEP 1: Manually Classify Based on long_descriptions that Include Judgements

This will serve as a high accuracy base line to train/test NLP models used later down the road:


In [5]:
# Define judgment-related keywords
judgment_keywords = ["judgment", "judgement"]

# Filter rows where any keyword appears in the combined_description column
judgment_mask = clean_docs['combined_description'].str.contains(
                      '|'.join(judgment_keywords), case=False, na=False)

# Resetting index to access columns by name
judgment_df = clean_docs[judgment_mask][['case_row_id', 'case_number',
                                         'doc_count', 'long_description',
                                         'combined_description']]


#first word should be "FINAL" - additional accuracy filter
clean_judgments = pd.DataFrame()
clean_judgments = judgment_df[judgment_df['long_description'
                                    ].str.split().str[0] == "FINAL"]

In [6]:
clean_judgments = clean_judgments.set_index(['case_row_id', 'case_number', 'doc_count']).sort_index()
clean_judgments

long_description  \
case_row_id case_number   doc_count                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
3.0         0:83-cv-06860 5                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           FINAL JUDGMENT for Kenneth R. Cornwall against U. S. Construction. Defendant permanently enjoined from manufacturing, advertising, etc. any device infringing patent; Permanent injunction effective as of date of entry of judgment; plaintiff shall recover from principal sum of $13,647.42 with pre-judgment interest from 4/30/85 thru 7/7/87 in the amount of $95.00 FOR WHICH LET EXECUTION ISSUE. ( signed by Judge Jose A. Gonzalez Jr. on 7/21/87) CCAP/M (Former Deputy Clerk) (Entered: 05/13/1991)   
53.0        0:91-cv-06658 13                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       FINAL JUDGMENT entered for Clotilde, Inc. and against Solar-Kist Corporation on the complaint and judgment is entered for defendant-counter-plaintiff on the counterclaim. Patents (4,320,699 and 4,597,812) are declared invalid. ( signed by Judge Kenneth L. Ryskamp on 9/8/93) CCAP/M (ea, Deputy Clerk) (Entered: 09/14/1993)   
72.0        0:93-cv-06063 3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               FINAL JUDGMENT Dismissi

Now we have a comprehensive dataframe containing only those files which we know
for sure are final judgments, maybe we can try using legalBERT on this for
classification of outcomes
---


In [7]:
pacer = pd.read_csv('/content/drive/MyDrive/ECO225_Final_Project/PART1/pacer_cases.csv')

In [8]:
pacer

,pacer_id,case_number,district_id,court_name,case_name,date_filed,date_closed,nos
0,NaN,2:04-cv-01438,azd,Arizona District Court,APM Design Lab Inc v. Rubicad Corporation,2004-07-14,NaN,NaN
1,NaN,2:04-cv-01438,azd,Arizona District Court,APM Design Lab Inc v. Rubicad Corporation,2004-07-14,NaN,NaN
2,NaN,2:05-at-03203,azd,Arizona District Court (Phoenix),"Test Advantage, Inc. v.",2005-10-25,NaN,NaN
3,NaN,1:03-cv-05889,cacd,California Central District Court,"La Leche League Intl v. Mothers Work, Inc.",2003-08-19,NaN,NaN
4,NaN,2:03-cv-03968,cacd,California Central District Court,"Amini Innovation v. Pacific Century Inc, et al",2003-06-04,NaN,NaN
...,...,...,...,...,...,...,...,...
96994,371335.0,3:20-cv-09482,cand,California Northern District Court,"Tekvoke, LLC v. Brekeke Software, Inc.",2020-12-31,2021-04-28,830.0
96995,371334.0,4:20-cv-09481,cand,California Northern District Court,Topdown Licensing LLC v. Guillemot Inc.,2020-12-31,2022-06-01,830.0
96996,806262.0,2:20-cv-11800,cacd,California Central District Court,"Tsteigen, Inc. et al v. Midwest Motor Supply Co.",2020-12-31,2021-03-18,830.0
96997,1118843.0,6:20-cv-01216,txwd,Texas Western District Court,"Ocean Semiconductor LLC v. Western Digital Technologies, Inc.",2020-12-31,2022-09-19,830.0


In [9]:
#adding pacer_id into clean_judgments for API based labelling of case outcomes
pacer_id = pacer[['case_number', 'pacer_id']]
clean_judgments = clean_judgments.merge(pacer_id, on='case_number', how='left')

In [10]:
clean_judgments

,case_number,long_description,combined_description,pacer_id
0,0:83-cv-06860,"FINAL JUDGMENT for Kenneth R. Cornwall against U. S. Construction. Defendant permanently enjoined from manufacturing, advertising, etc. any device infringing patent; Permanent injunction effective as of date of entry of judgment; plaintiff shall recover from principal sum of $13,647.42 with pre-judgment interest from 4/30/85 thru 7/7/87 in the amount of $95.00 FOR WHICH LET EXECUTION ISSUE. ( signed by Judge Jose A. Gonzalez Jr. on 7/21/87) CCAP/M (Former Deputy Clerk) (Entered: 05/13/1991)","FINAL JUDGMENT for Kenneth R. Cornwall against U. S. Construction. Defendant permanently enjoined from manufacturing, advertising, etc. any device infringing patent; Permanent injunction effective as of date of entry of judgment; plaintiff shall recover from principal sum of $13,647.42 with pre-judgment interest from 4/30/85 thru 7/7/87 in the amount of $95.00 FOR WHICH LET EXECUTION ISSUE. ( signed by Judge Jose A. Gonzalez Jr. on 7/21/87) CCAP/M (Former Deputy Clerk) (Entered: 05/13/1991)",27008.0
1,0:91-cv-06658,"FINAL JUDGMENT entered for Clotilde, Inc. and against Solar-Kist Corporation on the complaint and judgment is entered for defendant-counter-plaintiff on the counterclaim. Patents (4,320,699 and 4,597,812) are declared invalid. ( signed by Judge Kenneth L. Ryskamp on 9/8/93) CCAP/M (ea, Deputy Clerk) (Entered: 09/14/1993)","FINAL JUDGMENT entered for Clotilde, Inc. and against Solar-Kist Corporation on the complaint and judgment is entered for defendant-counter-plaintiff on the counterclaim. Patents (4,320,699 and 4,597,812) are declared invalid. ( signed by Judge Kenneth L. Ryskamp on 9/8/93) CCAP/M (ea, Deputy Clerk) (Entered: 09/14/1993)",34235.0
2,0:93-cv-06063,"FINAL JUDGMENT Dismissing case Status Conference set for 3/26/99 is Canceled (signed by Judge Jose A. Gonzalez Jr. on 3/4/99) CCAP (ss, Deputy Clerk) (Entered: 03/08/1999)","FINAL JUDGMENT Dismissing case Status Conference set for 3/26/99 is Canceled (signed by Judge Jose A. Gonzalez Jr. on 3/4/99) CCAP (ss, Deputy Clerk) (Entered: 03/08/1999)",38634.0
3,0:93-cv-06076,"FINAL JUDGMENT for Jaswant S. Pannu, Jaswant S. Pannu against IOLAB Corporation in the total amount of $670,667.47 ( signed by Judge Wilkie D. Ferguson Jr. on 6/12/97) CCAP/M (kp, Deputy Clerk) (Entered: 06/18/1997)","FINAL JUDGMENT for Jaswant S. Pannu, Jaswant S. Pannu against IOLAB Corporation in the total amount of $670,667.47 ( signed by Judge Wilkie D. Ferguson Jr. on 6/12/97) CCAP/M (kp, Deputy Clerk) (Entered: 06/18/1997)",38647.0
4,0:93-cv-06076,"FINAL JUDGMENT on jury verdict finding for IOLAB Corporation on claim for infringement by the Kelman 4-point lenses and the Small Kelman 3-point lenses. Judgment is entered for the plaintiffs on the claim for infringement by the large kelman 3-point lenses and both Bechart lenses. Judgment is entered for the defendant on the claim for willful infringement as to any of the defendant's intraocular lenses. Judgment is entered for the plaintiffs on the defendant's affirmative defenses of invalidity of the plaintiff's patent. Judgment is entered fthat the plaintiffs receive a reasonable royalty rate of 7% on the defendant's sales of the infringing lenses. Jurisdiction is reserved to (a) calculate the dollar amount of royalties based on the defendant's sales of the infringing lenses; (b) fashion an appropriate decree of injunction; and (c) determine an award reasonable fees and costs. ( signed by Judge Wilkie D. Ferguson Jr. on 2/10/97) CCAP/M (kp, Deputy Clerk) (Entered: 02/12/1997)","FINAL JUDGMENT on jury verdict finding for IOLAB Corporation on claim for infringement by the Kelman 4-point lenses and the Small Kelman 3-point lenses. Judgment is entered for the plaintiffs on the claim for infringement by the large kelman 3-point lenses and both Bechart lenses. Judgment is entered for the defendant on the claim for willful infringement as to any of the defendant's intraocular len

# Using RECAP API to get Case Outcomes

In [11]:
import requests
import time

# API Setup
API_KEY = "e3327ec94df6a3e5d37beaeb611fce27825c6c84"  # Your API Key
API_URL = "https://www.courtlistener.com/api/rest/v4/fjc-integrated-database/"
HEADERS = {"Authorization": f"Token {API_KEY}"}

# Load case numbers (limiting to first 50 for testing)
case_numbers = clean_judgments["case_number"].tolist()[:10]  # Limit to first 50 cases

# Function to fetch only the "Judgment" field
def get_judgment(case_number):
    params = {
    "docket_number": case_number,
    "pacer_case_id": pacer_id  }
    response = requests.get(API_URL, headers=HEADERS, params=params)

    if response.status_code == 200:
        data = response.json()
        if "results" in data and len(data["results"]) > 0:
            case_data = data["results"][0]

            # Debugging step: Print all keys to find the correct field name
            print(f"Available keys for {case_number}: {case_data.keys()}")

            # Try extracting "judgment" (if available) or test other keys
            judgment_value = case_data.get("judgment", case_data.get("nature_of_judgement", "Not Found"))
            return {"Case Number": case_number, "Judgment": judgment_value}

    return {"Case Number": case_number, "Judgment": "Not Found"}

# Run for only first 50 cases
results = []
for case in case_numbers:
    results.append(get_judgment(case))
    time.sleep(1)  # Prevent API rate limiting

# Convert to DataFrame & Save Results
df_results = pd.DataFrame(results)
df_results.to_csv("case_judgments_sample.csv", index=False)

print("✅ Judgment data retrieval complete for 50 cases. Saved to case_judgments_sample.csv")


Available keys for 0:83-cv-06860: dict_keys(['resource_uri', 'date_created', 'date_modified', 'dataset_source', 'office', 'docket_number', 'origin', 'date_filed', 'jurisdiction', 'nature_of_suit', 'title', 'section', 'subsection', 'diversity_of_residence', 'class_action', 'monetary_demand', 'county_of_residence', 'arbitration_at_filing', 'arbitration_at_termination', 'multidistrict_litigation_docket_number', 'plaintiff', 'defendant', 'date_transfer', 'transfer_office', 'transfer_docket_number', 'transfer_origin', 'date_terminated', 'termination_class_action_status', 'procedural_progress', 'disposition', 'nature_of_judgement', 'amount_received', 'judgment', 'pro_se', 'year_of_tape', 'nature_of_offense', 'version', 'circuit', 'district'])
Available keys for 0:91-cv-06658: dict_keys(['resource_uri', 'date_created', 'date_modified', 'dataset_source', 'office', 'docket_number', 'origin', 'date_filed', 'jurisdiction', 'nature_of_suit', 'title', 'section', 'subsection', 'diversity_of_residenc

In [12]:
print(requests.get("https://www.courtlistener.com/api/rest/v4/fjc-integrated-database/",
    headers={"Authorization": "e3327ec94df6a3e5d37beaeb611fce27825c6c84"},
    params={"docket_number": "0:91-cv-06658"}).json().get("results", [{}])[0].get("defendant", "Not Found"))

MUSIC TRIBE GLOBAL BRANDS LTD.


In [13]:
import requests

API_KEY = "e3327ec94df6a3e5d37beaeb611fce2785c6c84"
API_URL = "https://www.courtlistener.com/api/rest/v4/fjc-integrated-database/"
HEADERS = {"Authorization": f"Token {API_KEY}"}

case_number = "0:91-cv-06658"

response = requests.get(API_URL, headers=HEADERS, params={"docket_number": case_number})

if response.status_code == 200:
    data = response.json()
    print(f"\n🔹 API returned {data['count']} cases for {case_number}")
    for case in data.get("results", []):
        print(f"📌 Docket: {case.get('docket_number')} | Filed: {case.get('date_filed')} | Court: {case.get('district')} | Judgment: {case.get('judgment', 'Not Found')}")
else:
    print(f"⚠️ API request failed, Status Code: {response.status_code}")


⚠️ API request failed, Status Code: 401


In [14]:
JFC_1970_88 = pd.read_csv('/content/drive/MyDrive/ECO225_Final_Project/PART1/Civil_1970_to_1987.txt', sep='\t')

<ipython-input-14-1b630a61f580>:1: DtypeWarning: Columns (1,2,3,14) have mixed types. Specify dtype option on import or set low_memory=False.
  JFC_1970_88 = pd.read_csv('/content/drive/MyDrive/ECO225_Final_Project/PART1/Civil_1970_to_1987.txt', sep='\t')


In [15]:
JFC_1970_88.columns

Index(['CIRCUIT', 'DISTRICT', 'OFFICE', 'DOCKET', 'FILEDATE', 'JURIS', 'NOS',
       'ORIGIN', 'RESIDENC', 'CLASSACT', 'TERMJUDG', 'FILEJUDG', 'DEMANDED',
       'FILEMAG', 'COUNTY', 'CASENAME', 'TERMDATE', 'FDATEUSE', 'DISP',
       'TERMMAG', 'PROCPROG', 'NOJ', 'AMTREC', 'JUDGMENT', 'MAGISINV',
       'OTHERINV', 'TDATEUSE', 'TAPEYEAR'],
      dtype='object')

In [16]:
documents.head()

,case_row_id,case_number,case_number_raw,district_id,doc_count,doc_number,short_description,long_description,attachment,doc_date_filed,doc_date_uploaded,document_url
0,1.0,0:79-cv-06704,0:79-cv-06704-JCP,flsd,1,37.0,NaN,"COPY OF PAPER DOCKET SHEET (kw, Deputy Clerk) (Entered: 08/03/2000)",NaN,2000-08-03,NaN,NaN
1,1.0,0:79-cv-06704,0:79-cv-06704-JCP,flsd,2,NaN,NaN,"CASE CLOSED. Case and Motions no longer referred to Magistrate. (kw, Deputy Clerk) (Entered: 08/03/2000)",NaN,1982-05-31,NaN,NaN
2,3.0,0:83-cv-06860,0:83-cv-06860-JAG,flsd,1,123.0,NaN,COPY OF PAPER DOCKET SHEET (Former Deputy Clerk) (Entered: 02/13/2004),NaN,2004-02-13,NaN,NaN
3,3.0,0:83-cv-06860,0:83-cv-06860-JAG,flsd,2,NaN,NaN,Case closed (Former Deputy Clerk) (Entered: 03/05/1992),NaN,1992-03-01,NaN,NaN
4,3.0,0:83-cv-06860,0:83-cv-06860-JAG,flsd,3,NaN,NaN,Case reopened (Former Deputy Clerk) (Entered: 05/13/1991),NaN,1990-10-16,NaN,NaN


In [18]:
JFC_1970_88['case_number'] = JFC_1970_88['DOCKET'].astype(str).str[:2] + "-cv-" + JFC_1970_88['DOCKET'].astype(str).str[2:]

In [21]:
JFC_1970_88.head()

,CIRCUIT,DISTRICT,OFFICE,DOCKET,FILEDATE,JURIS,NOS,ORIGIN,RESIDENC,CLASSACT,...,TERMMAG,PROCPROG,NOJ,AMTREC,JUDGMENT,MAGISINV,OTHERINV,TDATEUSE,TAPEYEAR,case_number
0,0,90,1,6900239,11/07/1977,1,370,4,-8,0,...,NaN,3,-8,0,4,9,5,8505,1985,69-cv-00239
1,0,90,1,6903291,10/29/1973,3,830,3,-8,0,...,NaN,4,-8,0,4,9,5,8505,1985,69-cv-03291
2,0,90,1,7100659,11/07/1977,1,370,4,-8,0,...,NaN,3,-8,0,4,9,5,8505,1985,71-cv-00659
3,0,90,1,7201788,09/06/1972,2,520,1,-8,0,...,NaN,4,0,0,4,9,5,8505,1985,72-cv-01788
4,0,90,1,7201836,09/13/1972,2,520,1,-8,0,...,NaN,4,3,0,4,9,5,8505,1985,72-cv-01836


In [19]:
cases = pd.read_csv('/content/drive/MyDrive/ECO225_Final_Project/PART1/cases.csv')

In [20]:
cases

,case_row_id,case_number,case_number_raw,district_id,court_name,pacer_id,case_name,assigned_to,referred_to,case_cause,...,date_last_filed,jury_demand,demand,lead_case,related_case,settlement,case_type_1,case_type_2,case_type_3,case_type_note
0,78028.0,4:92-cv-00055,4:92-cv-00055-JWS,akd,Alaska District Court,5408.0,"Melba Creek Mining, Inc. et al v. Sykes et al",Judge John W Sedwick,NaN,No cause code entered,...,1992-12-22,Defendant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,75332.0,2:10-cv-00601,2:10-cv-00601-WKW-TFM,almd,Middle District of Alabama,43636.0,Bowhall v. Johnson & Johnson Inc. et al (MAG+),Honorable William Keith Watkins,Honorable Terry F. Moorer,15:1126 Patent Infringement,...,2011-03-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6908.0,1:98-cv-00596,1:98-cv-00596-RBP,alnd,Northern District of Alabama,5576.0,Mettke v. Touchnet Information,Judge Robert B Propst,NaN,35:183 Patent Infringement,...,1998-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6933.0,1:98-cv-00654,1:98-cv-00654-CB-C,alsd,Southern District of Alabama,27451.0,"Monsanto Co. Inc. v. Styron, et al","Senior Judge Charles R. Butler, Jr",Magistrate Judge William E. Cassady,28:1338 Patent Infringement,...,1999-03-15,Defendant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,86730.0,1:18-cv-00012,1:18-cv-00012-BRW,ared,Eastern District of Arkansas,110647.0,Sundesa LLC v. Eurark LLC,Judge Billy Roy Wilson,NaN,35:271 Patent Infringement,...,2018-07-06,Both,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96961,29713.0,2:97-cv-00019,2:97-cv-00019-ABJ,wyd,Wyoming District Court,8221.0,Chey Land & Cattle v. Crown Cork & Seal Co,Honorable Alan B Johnson,NaN,35:271 Patent Infringement,...,1997-07-25,Plaintiff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96962,39890.0,2:11-cv-00352,2:11-cv-00352-SWS,wyd,Wyoming District Court,23426.0,Weir Minerals Australia Ltd v. Megatrol Inc et al,Honorable Scott W Skavdahl,Honorable Scott W Skavdahl,35:271 Patent Infringement,...,2012-11-15,Both,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
96963,104380.0,2:15-cv-00149,2:15-cv-00149-ABJ,wyd,Wyoming District Court,33849.0,Kims Kreations Inc v. Walt Disney Company,Judge Alan B Johnson,Honorable Kelly H Rankin,28:1331 Fed. Question: Trademark,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96964,37186.0,2:08-cv-00045,2:08-cv-00045-WFD,wyd,Wyoming District Court,17465.0,Retractable Technologies LLC v. RadioShack Corporation et al,Judge William F Downes,Judge William C Beaman,35:271 Patent Infringement,...,2009-07-20,Both,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


In [ ]:
JFC_1988_Present = pd.read_csv('/content/drive/MyDrive/ECO225_Final_Project/PART1/Civil_1988_to_Present.txt', sep='\t', on_bad_lines='skip', encoding='latin-1') # or 'cp1252'

In [ ]:
JFC_1988_Present